In [1]:
pip install us

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import us as us

import io
from zipfile import ZipFile

In [12]:
# raw = pd.read_csv('data/cps_00001.csv')

# raw_2 = pd.read_csv('http://localhost:8888/edit/UBICenter/min-wage/jb/data/cps_00002.csv.gz')

raw_2 = pd.read_csv('UBICenter/min-wage/jb/data/cps_00002.csv.gz')


FileNotFoundError: [Errno 2] No such file or directory: 'UBICenter/min-wage/jb/data/cps_00002.csv.gz'

Only include 2018-2020 ASEC, then monthly data for 2020. Need year and month, hourwage, state and county (need that), diffany (disability), citizenship, spm_family_unit, spm resources, spm pov threshold

In [13]:
raw_2

NameError: name 'raw_2' is not defined

In [20]:
raw_2.HOURWAGE.describe()

count    2.027648e+06
mean     9.466096e+02
std      2.224880e+02
min      4.000000e-01
25%      9.999900e+02
50%      9.999900e+02
75%      9.999900e+02
max      9.999900e+02
Name: HOURWAGE, dtype: float64

In [39]:
min_wage_data = raw_2[raw_2.HOURWAGE < 999.99]

# check IPUMS field documentation
# make sure to copy dataset before starting

min_wage_data.HOURWAGE.describe()

count    110380.000000
mean         19.408039
std          10.921556
min           0.400000
25%          12.500000
50%          16.000000
75%          23.000000
max          99.990000
Name: HOURWAGE, dtype: float64

In [40]:
min_wage_data['under_15'] = min_wage_data.HOURWAGE < 15


<ipython-input-40-93266fefc1c0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_wage_data['under_15'] = min_wage_data.HOURWAGE < 15


38         False
71          True
88          True
220         True
221        False
           ...  
2027559    False
2027560     True
2027616    False
2027631    False
2027632    False
Name: under_15, Length: 110380, dtype: bool

In [43]:
min_wage_data['u15_t'] = np.where(min_wage_data.under_15 == True, 1, 0)

min_wage_data['u15_f'] = np.where(min_wage_data.under_15 == False, 1, 0)




<ipython-input-43-543a574579f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_wage_data['u15_t'] = np.where(min_wage_data.under_15 == True, 1, 0)
<ipython-input-43-543a574579f4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_wage_data['u15_f'] = np.where(min_wage_data.under_15 == False, 1, 0)


,YEAR,SERIAL,MONTH,HWTFINL,CPSID,ASECFLAG,ASECWTH,STATEFIP,COUNTY,PERNUM,...,SPMTOTRES,SPMNADULTS,SPMNCHILD,SPMNPERS,SPMTHRESH,SPMFAMUNIT,HOURWAGE,under_15,u15_t,u15_f
38,2018,26,3,NaN,20171202868100,1.0,1120.98,23,0,2,...,60061.008,2.0,0.0,2.0,13632.36586,26001.0,16.00,False,0,1
71,2018,40,3,NaN,20171202867300,1.0,716.67,23,0,1,...,399.232,1.0,3.0,4.0,22563.73248,40001.0,9.00,True,1,0
88,2018,46,3,NaN,20161202950000,1.0,1153.61,23,0,1,...,19710.008,1.0,0.0,1.0,10974.89364,46001.0,7.50,True,1,0
220,2018,143,3,NaN,20171202879300,1.0,1266.70,23,0,1,...,54334.016,2.0,0.0,2.0,15474.60003,143001.0,12.50,True,1,0
221,2018,143,3,NaN,20171202879300,1.0,1266.70,23,0,2,...,54334.016,2.0,0.0,2.0,15474.60003,143001.0,50.00,False,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027559,2020,68876,12,369.0110,20190906988000,NaN,NaN,56,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,21.00,False,0,1
2027560,2020,68876,12,369.0110,20190906988000,NaN,NaN,56,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,12.00,True,1,0
2027616,2020,68904,12,381.7554,20190906991200,NaN,NaN,56,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,35.00,False,0,1
2027631,2020,68920,12,254.8724,20200906860600,NaN,NaN,56,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,32.25,False,0,1


In [14]:
mw_gby_state = min_wage_data.groupby(['STATEFIP']).sum()


NameError: name 'min_wage_data' is not defined

In [9]:
mw_gby_state['num_u15'] = mw_gby_state.u15_t * mw_gby_state.WTFINL

mw_gby_state['num_u15_f'] = mw_gby_state.u15_f * mw_gby_state.WTFINL

mw_gby_state['pct_below15'] = mw_gby_state.num_u15/(mw_gby_state.num_u15 + mw_gby_state.num_u15_f)


NameError: name 'mw_gby_state' is not defined

In [6]:
def read_zip(zip_fn, extract_fn=None):
    zf = ZipFile(zip_fn)
    if extract_fn:
        return zf.read(extract_fn)
    else:
        return {name:zf.read(name) for name in zf.namelist()}


raw = pd.read_excel(io.BytesIO(read_zip('data/oes_2019.xlsx.zip', 'all_data_M_2019.xlsx')))

raw

 # UBICenter/min-wage/jb/data/updated_ds.csv.gz

,area,area_title,area_type,naics,naics_title,i_group,own_code,occ_code,occ_title,o_group,...,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,annual,hourly
0,99,U.S.,1,000000,Cross-industry,cross-industry,1235,11-0000,Management Occupations,major,...,50.8,74.16,#,49990,71460,105660,154260,#,NaN,NaN
1,99,U.S.,1,000000,Cross-industry,cross-industry,1235,13-0000,Business and Financial Operations Occupations,major,...,33.57,45.61,60.6,39020,52130,69820,94870,126040,NaN,NaN
2,99,U.S.,1,000000,Cross-industry,cross-industry,1235,15-0000,Computer and Mathematical Occupations,major,...,42.47,57.47,73.08,45320,62850,88340,119550,152010,NaN,NaN
3,99,U.S.,1,000000,Cross-industry,cross-industry,1235,17-0000,Architecture and Engineering Occupations,major,...,39.15,52.87,68.56,45280,60910,81440,109970,142610,NaN,NaN
4,99,U.S.,1,000000,Cross-industry,cross-industry,1235,19-0000,"Life, Physical, and Social Science Occupations",major,...,32.77,46.24,61.59,36640,49360,68160,96180,128100,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395642,99,U.S.,1,622100,General Medical and Surgical Hospitals,"4-digit, ownership",5,00-0000,All Occupations,total,...,28.4,39.98,55.78,28450,38120,59070,83160,116020,NaN,NaN
395643,99,U.S.,1,622200,Psychiatric and Substance Abuse Hospitals,"4-digit, ownership",2,00-0000,All Occupations,total,...,22.68,33.58,47.09,25300,32950,47180,69850,97940,NaN,NaN
395644,99,U.S.,1,622200,Psychiatric and Substance Abuse Hospitals,"4-digit, ownership",5,00-0000,All Occupations,total,...,21.5,31.28,43.38,25570,31680,44720,65060,90240,NaN,NaN
395645,99,U.S.,1,622300,Specialty (except Psychiatric and Substance Ab...,"4-digit, ownership",2,00-0000,All Occupations,total,...,30.05,45.08,63.88,29070,39850,62490,93770,132870,NaN,NaN


In [7]:
# only want hourly incomes, percentiles
# want job code to be 00-0000
# what percentage of poeple are in counties/other areas where median income is below 15?\


bls_data = raw.copy(deep=True)

bls_data


print(bls_data.columns.tolist())

['area', 'area_title', 'area_type', 'naics', 'naics_title', 'i_group', 'own_code', 'occ_code', 'occ_title', 'o_group', 'tot_emp', 'emp_prse', 'jobs_1000_orig', 'loc_quotient', 'pct_total', 'h_mean', 'a_mean', 'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90', 'a_pct10', 'a_pct25', 'a_median', 'a_pct75', 'a_pct90', 'annual', 'hourly']


In [8]:
# select only the necessary columns in the dataset

bls_data = bls_data[['area', 'area_type', 'area_title', 'naics', 'occ_code', 'tot_emp', 
                     'h_mean', 'h_median', 'h_pct25', 'h_pct75']]

In [9]:
bls_data_t = bls_data[bls_data.occ_code == '00-0000']

In [10]:
# group by each area level first
# mw_gby_state = min_wage_data.groupby(['STATEFIP']).sum()

bls_area_states = bls_data_t[bls_data_t.area_type == 2]

bls_area_territories = bls_data_t[bls_data_t.area_type == 3]

bls_area_city = bls_data_t[bls_data_t.area_type == 4]

bls_area_metro_area = bls_data_t[bls_data_t.area_type == 6]

In [12]:
bls_area_states
# no state has a median below $15/hour

,area,area_type,area_title,naics,occ_code,tot_emp,h_mean,h_median,h_pct25,h_pct75
394608,1,2,Alabama,000000,00-0000,1974170,21.6,16.73,11.06,26.34
394609,2,2,Alaska,000000,00-0000,317090,28.51,23.34,15.57,35.81
394610,4,2,Arizona,000000,00-0000,2866820,24.49,18.46,12.99,28.95
394611,5,2,Arkansas,000000,00-0000,1217420,20.52,15.84,11.18,24.26
394612,6,2,California,000000,00-0000,17382400,29.47,21.24,14.17,36.44
394613,8,2,Colorado,000000,00-0000,2678490,27.73,21.28,14.31,33.65
394614,9,2,Connecticut,000000,00-0000,1665100,29.98,23.33,14.71,37.23
394615,10,2,Delaware,000000,00-0000,451900,26.14,19.66,12.95,31.95
394616,11,2,District of Columbia,000000,00-0000,723510,43.17,35.74,20.76,57.64
394617,12,2,Florida,000000,00-0000,8794050,22.96,17.23,11.87,27.26


In [13]:
bls_area_territories

# medians are 13.26 for Guam and 10.13 for Puerto Rico

,area,area_type,area_title,naics,occ_code,tot_emp,h_mean,h_median,h_pct25,h_pct75
394659,66,3,Guam,000000,00-0000,63630,17.75,13.26,9.5,21.27
394660,72,3,Puerto Rico,000000,00-0000,865160,14.27,10.13,8.63,16.03
394661,78,3,Virgin Islands,000000,00-0000,34420,22.1,17.33,12.17,26.29


In [18]:
bls_area_city['median_under_15'] = bls_area_city.h_median < 15

bls_area_city['median_under_15'].value_counts()

# 7.56% of cities have a median income below $15/hour

<ipython-input-18-721452392e07>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bls_area_city['median_under_15'] = bls_area_city.h_median < 15


False    366
True      30
Name: median_under_15, dtype: int64

In [21]:
bls_area_metro_area['median_under_15'] = bls_area_metro_area.h_median < 15

bls_area_metro_area['median_under_15'].value_counts()

# 22.39% of Metro areas have a median income below $15/hour

<ipython-input-21-b5eb32a9c7bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bls_area_metro_area['median_under_15'] = bls_area_metro_area.h_median < 15


False    104
True      30
Name: median_under_15, dtype: int64